Author: [Zhedong Zheng](https://github.com/zhedongzheng)

![title](img/text_cnn.png)

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report

In [2]:
VOCAB_SIZE = 5000
MAX_LEN = 400
BATCH_SIZE = 32
EMBED_DIM = 50
FILTERS = 250
N_CLASS = 2
N_EPOCH = 2
LR = {'start': 5e-3, 'end': 5e-4, 'steps': 1500}

In [3]:
def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, VOCAB_SIZE, EMBED_DIM)
        x = tf.layers.dropout(x, 0.2, training=is_training)
        feat_map = []
        for k_size in [3, 4, 5]:
            _x = tf.layers.conv1d(x, FILTERS, k_size, activation=tf.nn.relu)
            _x = tf.layers.max_pooling1d(_x, _x.get_shape().as_list()[1], 1)
            _x = tf.reshape(_x, (tf.shape(x)[0], FILTERS))
            feat_map.append(_x)
        x = tf.concat(feat_map, -1)
        x = tf.layers.dense(x, FILTERS, tf.nn.relu)
        logits = tf.layers.dense(x, N_CLASS)
    return logits


def model_fn(features, labels, mode, params):
    logits = forward(features, reuse=False, is_training=True)
    preds = forward(features, reuse=True, is_training=False)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        preds = tf.argmax(preds, -1)
        return tf.estimator.EstimatorSpec(mode, predictions=preds)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step = tf.train.get_global_step()

        lr_op = tf.train.exponential_decay(
            LR['start'], global_step, LR['steps'], LR['end']/LR['start'])

        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

        train_op = tf.train.AdamOptimizer(lr_op).minimize(
            loss_op, global_step=global_step)

        lth = tf.train.LoggingTensorHook({'lr': lr_op}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[lth])

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, MAX_LEN)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, MAX_LEN)

estimator = tf.estimator.Estimator(model_fn)

for _ in range(N_EPOCH):
    estimator.train(tf.estimator.inputs.numpy_input_fn(
        x = X_train, y = y_train,
        batch_size = BATCH_SIZE,
        shuffle = True))
    y_pred = np.fromiter(estimator.predict(tf.estimator.inputs.numpy_input_fn(
        x = X_test,
        batch_size = BATCH_SIZE,
        shuffle = False)), np.int32)
    print("\nValidation Accuracy: %.4f\n" % (y_pred==y_test).mean())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpccz77f65', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120f57be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:t